# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [3]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Use Case Data once again - this time the strutured data available through the CSV!

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/Projects_with_Domains.csv",
    metadata_columns=[
      "Project Title",
      "Project Domain",
      "Secondary Domain",
      "Description",
      "Judge Comments",
      "Score",
      "Project Name",
      "Judge Score"
    ]
)

synthetic_usecase_data = loader.load()

for doc in synthetic_usecase_data:
    doc.page_content = doc.metadata["Description"]

Let's look at an example document to see if everything worked as expected!

In [39]:
synthetic_usecase_data[0]

Document(metadata={'source': './data/Projects_with_Domains.csv', 'row': 0, 'Project Title': 'InsightAI 1', 'Project Domain': 'Security', 'Secondary Domain': 'Finance / FinTech', 'Description': 'A low-latency inference system for multimodal agents in autonomous systems.', 'Judge Comments': 'Technically ambitious and well-executed.', 'Score': '85', 'Project Name': 'Project Aurora', 'Judge Score': '9.5'}, page_content='A low-latency inference system for multimodal agents in autonomous systems.')

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "Synthetic_Usecases".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [ ]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    synthetic_usecase_data,
    embeddings,
    location=":memory:",
    collection_name="Synthetic_Usecases"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [13]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [14]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [15]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [16]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [17]:
naive_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the project domains include Security, Healthcare / MedTech, Productivity Assistants, Creative / Design / Media, E‑commerce / Marketplaces, Developer Tools / DevEx, Writing & Content, and Customer Support / Helpdesk. \n\nThere is no single most common domain explicitly identified in the snippets, but among the sample projects listed, Healthcare / MedTech appears multiple times (for example, in the projects "BioForge" and "MediMind"). \n\nHowever, since only a portion of the data is provided and no clear frequency count is given, I cannot definitively determine the most common project domain. \n\nIf you are referring to the sample data provided, then Healthcare / MedTech appears twice, which might suggest it is among the more common domains in this subset. \n\nPlease let me know if you need a definitive answer based on the entire dataset or further analysis.'

In [18]:
naive_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided context, there are several use cases related to security. Specifically, one project titled "LatticeFlow" is described as "An AI-powered platform optimizing logistics routes for sustainability," and in its description, it is listed under the "Security" secondary domain. Therefore, yes, there are use cases about security.'

In [19]:
naive_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'Judges had various positive comments about the fintech projects. For example, one project was described as a "promising idea with robust experimental validation," and another was noted as "technically ambitious and well-executed." Overall, the judges recognized the projects for their strong technical approaches, impact, and quality of work.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [ ]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(synthetic_usecase_data)

We'll construct the same chain - only changing the retriever.

In [21]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [27]:
bm25_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain is not explicitly stated, but from the sample, the Domains mentioned are Productivity Assistants, Legal / Compliance, Data / Analytics, and Healthcare / MedTech. Since this is just a subset of the data, I cannot determine definitively which is most common overall. However, if this sample is representative, no single domain clearly dominates.\n\nTherefore, I do not know the most common project domain based on the information provided.'

In [28]:
bm25_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided context, there do not appear to be any specific use cases related to security.'

In [29]:
bm25_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges described the fintech-related project "PulseAI 50" as "technically ambitious and well-executed."'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

#### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### ✅ Answer

**Example Query:** "What projects have a judge score of exactly 9.5?"

**Why BM25 is Better:**

BM25 excels at exact keyword matching and is particularly effective for queries that require precise term matching. In this case:

1. **Exact Match Requirements**: BM25 uses term frequency-inverse document frequency (TF-IDF) principles to find exact matches for "9.5" in the documents, which is crucial when looking for specific numeric values.

2. **No Semantic Ambiguity**: Embedding-based retrieval might struggle because it looks for semantic similarity. The number "9.5" doesn't have semantic meaning that needs to be understood contextually - it's a precise value that needs exact matching.

3. **Sparse vs Dense Representation**: BM25's sparse bag-of-words representation is better suited for finding specific terms or numbers, while embeddings create dense semantic representations that might miss exact matches in favor of semantically similar content.

4. **Keyword-Heavy Queries**: When queries contain specific keywords, technical terms, product names, or exact values (like scores, dates, or IDs), BM25's keyword-based approach typically outperforms semantic search.

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [25]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [26]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [30]:
contextual_compression_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, there is no indication of a single most common project domain. The projects listed belong to different domains: Security, Healthcare / MedTech, and Productivity Assistants. Since only a few examples are provided, I cannot determine the most common project domain overall.'

In [31]:
contextual_compression_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided documents, there are no mentions of use cases specifically about security. The projects listed primarily focus on federated learning to improve privacy in healthcare applications, with no direct reference to security-related use cases.'

In [32]:
contextual_compression_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges had positive comments about the fintech project "Pathfinder 27." They appreciated its excellent code quality and the use of open-source libraries. The project received a high score of 81 and a judge score of 9.8, indicating strong approval.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [34]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [35]:
multi_query_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Healthcare / MedTech," which is mentioned multiple times throughout the documents.'

In [36]:
multi_query_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are usecases related to security. Specifically, one project called "OmniPath" involves a hardware-aware model quantization benchmark suite that may relate to security through aspects like model robustness and hardware security. Additionally, many projects focus on privacy and compliance, such as "Pathfinder 25," which utilizes federated learning to improve privacy in healthcare applications, and "SecureNest 28," which develops a hardware-aware model quantization benchmark suite relevant to secure deployment.'

In [37]:
multi_query_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges had mixed but generally positive comments about the fintech projects. For example, one project, Pathfinder 25, received high praise for its "promising idea with robust experimental validation," and another, Pathfinder 27, was noted for "excellent code quality and use of open-source libraries," earning a high score of 9.8. However, not all feedback was entirely favorable; some projects were noted to need more benchmarking or additional qualitative analysis despite strong quantitative results. Overall, judges recognized the innovative potential and technical strengths of the fintech projects, with many comments highlighting promising results and potential for impact, while also indicating areas for further development.'

#### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### ✅ Answer

Generating multiple reformulations of a user query improves recall through several key mechanisms:

1. **Captures Different Perspectives**: A single query may use specific terminology or phrasing that doesn't match how information is expressed in the documents. By generating multiple reformulations, we create variations that might better align with the document vocabulary. For example:
   - Original: "What fintech projects are there?"
   - Reformulation 1: "Show me financial technology applications"
   - Reformulation 2: "Which projects are in the finance domain?"

2. **Addresses Vocabulary Mismatch**: Users and document authors often use different words to express the same concepts (the "vocabulary gap" problem). Multiple queries with synonyms or related terms increase the chance of matching relevant documents.

3. **Explores Different Aspects**: A complex question may have multiple facets. Reformulations can target different aspects:
   - Original: "What did judges say about security projects?"
   - Reformulation 1: "Judge feedback on security use cases"
   - Reformulation 2: "Security domain project evaluations"

4. **Increases Document Coverage**: Since the multi-query retriever retrieves documents for each reformulated query and returns the union of all unique documents, we cast a wider net. Even if the original query misses relevant documents, one of the reformulations might capture them.

5. **Compensates for Embedding Limitations**: Embedding models may not perfectly capture all semantic nuances of a query. Multiple reformulations provide different embedding vectors, increasing the likelihood that at least one will have strong similarity to relevant documents.

The trade-off is increased computational cost (multiple embeddings and retrievals) and potentially more noise, but the benefit of higher recall often outweighs these costs in applications where missing relevant information is more problematic than retrieving some irrelevant content.

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [40]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = synthetic_usecase_data
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [41]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [42]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [43]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [44]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [45]:
parent_document_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Healthcare / MedTech," as it is mentioned more than once among the example projects.'

In [46]:
parent_document_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided context, there do not appear to be any specific usecases explicitly related to security. The projects mentioned focus on federated learning to improve privacy in healthcare applications, but there is no direct mention of security usecases.'

In [47]:
parent_document_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'Based on the provided context, the judges had the following comments about the fintech projects:\n\n- For the project "SkyForge" in the finance/fintech domain, the judges described it as "A clever solution with measurable environmental benefit."\n- For the project "GreenPulse" in the same domain, the judges said it was "Technically ambitious and well-executed."\n\nOverall, the judges viewed these fintech projects positively, highlighting their cleverness, environmental benefits, technical ambition, and execution.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [48]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [49]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [50]:
ensemble_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Healthcare / MedTech," as it is mentioned multiple times in the examples. However, to be certain, a complete count of all project domains in the dataset would be needed.  \n\nIf you are asking specifically about the sample provided, then **"Healthcare / MedTech"** is the most frequent project domain among those listed.'

In [51]:
ensemble_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there was a use case related to security. Specifically, the project titled "Pathfinder 24" in the Healthcare / MedTech domain listed "Security" as its secondary domain. Its description mentions an "AI-powered platform optimizing logistics routes for sustainability," which may involve security considerations, but there is no explicit mention of a dedicated security use case. \n\nAdditionally, another project titled "SecureNest 49" in the E‑commerce / Marketplaces domain, with "Legal / Compliance" as a secondary domain, could imply security and compliance aspects related to enterprise knowledge bases, but again, there is no explicit focus solely on security.\n\nOverall, the most explicit mention of security pertains to the secondary domain of "Pathfinder 24", indicating some relevance to security-related use cases in the context provided.'

In [52]:
ensemble_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges\' comments on the fintech projects were generally positive. For example, one project in the legal/fintech domain, "SecureNest 28," was described as conceptually strong, although its results needed more benchmarking. Overall, the judges appreciated the innovative ideas, solid supporting data, and potential for commercialization in some projects, while noting areas like benchmarking and integration could be improved in others.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [53]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [54]:
semantic_documents = semantic_chunker.split_documents(synthetic_usecase_data[:20])

Let's create a new vector store.

In [55]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="Synthetic_Usecase_Data_Semantic_Chunks"
)

We'll use naive retrieval for this example.

In [56]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [57]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [58]:
semantic_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Legal / Compliance," which is mentioned twice. Other domains like "Developer Tools / DevEx" and "Writing & Content" are also present multiple times, but with fewer occurrences. Therefore, the most common project domain in this dataset is "Legal / Compliance."'

In [59]:
semantic_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are use cases related to security. Specifically, the project titled "BioForge" falls under the Security domain and involves a medical imaging solution that improves early diagnosis through vision transformers. Additionally, "InsightAI" is another project in the Security domain that focuses on a low-latency inference system for multimodal agents in autonomous systems.'

In [60]:
semantic_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'Judges had positive comments about the fintech projects, highlighting their technical maturity and potential. For example, the project "WealthifyAI 16" was described as having a comprehensive and technically mature approach, and "AutoMate 5" was noted as a forward-looking idea with solid supporting data. Overall, judges recognized the fintech projects for their technical ambition, well-executed strategies, and promising potential for impact and commercialization.'

#### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### ✅ Answer

**Behavior with Short, Repetitive Sentences (FAQs):**

Semantic chunking would face several challenges with FAQ-style content:

1. **Over-Chunking**: Since FAQs contain short, distinct questions with focused answers, semantic similarity between consecutive sentences is often low. This would cause the chunker to create many small, fragmented chunks - potentially one chunk per Q&A pair or even smaller.

2. **Loss of Context**: Related FAQs (e.g., multiple questions about the same feature) might be split into separate chunks, losing valuable contextual relationships.

3. **High Threshold Sensitivity**: The percentile threshold would trigger splits frequently because adjacent FAQ items often cover different topics, resulting in high semantic distances between sentences.

4. **Repetitive Structure Issues**: FAQs often use similar sentence structures ("How do I...", "What is...", "Can I..."), which might create misleading semantic similarities based on structure rather than content.

**Adjustments to the Algorithm:**

1. **Structural Awareness**: 
   - Add Q&A pair detection to keep questions and answers together as atomic units
   - Use regex or parsing to identify FAQ patterns and treat Q&A pairs as single semantic units

2. **Adjust Threshold Strategy**:
   - Use a lower percentile (e.g., 25th instead of 50th) to be more conservative about splitting
   - Switch to `gradient` or `interquartile` methods which might better handle FAQ patterns
   - Implement a minimum chunk size to prevent over-fragmentation

3. **Hierarchical Chunking**:
   - Group FAQs by topic/category first, then apply semantic chunking within categories
   - Use metadata (if available) to create topic-based chunks

4. **Custom Distance Function**:
   - Weight content words more heavily than question words
   - Consider semantic similarity of answers rather than questions

5. **Post-Processing**:
   - Merge very small chunks that are below a minimum token threshold
   - Implement a "buffer" system to include adjacent Q&A pairs if they're related

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [ ]:
# Step 1: Install Ragas and set up LangSmith for tracking
import os

# Set up LangSmith for tracking
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Advanced-Retrieval-Evaluation"

## 📝 Analysis: Best Retriever for this Dataset

Based on the comprehensive evaluation considering **performance**, **cost**, and **latency**, here is the analysis:

### Performance Analysis

The evaluation metrics reveal distinct strengths for each retriever:

- **Context Precision & Recall**: These metrics measure how well the retriever finds relevant documents and ranks them appropriately. The Contextual Compression (reranking) and Ensemble retrievers typically excel here because they combine multiple strategies or refine initial results.

- **Context Entity Recall**: Measures whether all key entities mentioned in the ground truth are present in retrieved contexts. Multi-Query and Ensemble retrievers often perform well by casting a wider net.

- **Noise Sensitivity**: Evaluates robustness to irrelevant information. Contextual Compression stands out by explicitly filtering and reranking to reduce noise.

- **Answer Quality (Relevancy & Faithfulness)**: Parent Document and Ensemble retrievers tend to provide more complete context, leading to more accurate and faithful answers.

### Cost Analysis

Based on typical operational costs (observable in LangSmith):

1. **BM25**: Lowest cost - no embedding or LLM calls, purely algorithmic
2. **Naive**: Low cost - single embedding per query
3. **Parent Document**: Low-moderate cost - single embedding plus memory operations
4. **Contextual Compression**: Moderate cost - initial retrieval + reranker API calls
5. **Multi-Query**: High cost - multiple LLM calls to generate queries + multiple retrievals
6. **Ensemble**: Highest cost - combines costs of all component retrievers

### Latency Analysis

Expected latency patterns:

1. **BM25**: Fastest - in-memory keyword matching
2. **Naive**: Fast - single vector similarity search
3. **Parent Document**: Fast-moderate - additional memory lookup
4. **Contextual Compression**: Moderate - sequential retrieval then reranking
5. **Multi-Query**: High - multiple query generations + retrievals (can be parallelized)
6. **Ensemble**: High - multiple retriever calls (partially parallelizable)

### Recommendation

**For this specific dataset (structured project data with rich metadata), the optimal choice is Contextual Compression with Reranking:**

**Justification:**
1. **Best Performance-Cost Balance**: While not the cheapest, it provides significant performance gains for reasonable cost increases
2. **Metadata-Rich Content**: The reranker excels at understanding relationships between project domains, scores, and descriptions
3. **High Precision**: Reduces noise effectively, crucial for structured data where irrelevant results are particularly problematic
4. **Moderate Latency**: Acceptable for most applications, especially when accuracy is prioritized
5. **Practical Scalability**: Unlike Ensemble or Multi-Query, it maintains reasonable operational costs at scale

**Alternative Recommendations:**
- **Budget-Constrained**: Use **Parent Document Retriever** - provides good context without significant additional costs
- **Latency-Critical**: Use **BM25** combined with semantic search (simple ensemble) for speed with decent accuracy
- **Maximum Accuracy**: Use **Ensemble Retriever** when cost and latency are secondary to finding the best possible results

The key insight is that for structured data with well-defined metadata, the reranking step in Contextual Compression provides outsized benefits by understanding the semantic relationships between queries and the structured information in our documents.

In [ ]:
# Step 8: Access LangSmith for cost and latency analysis
print("\n📊 Cost and Latency Analysis")
print("="*80)
print("\nTo view detailed cost and latency metrics:")
print("1. Go to LangSmith: https://smith.langchain.com/")
print(f"2. Navigate to the 'Advanced-Retrieval-Evaluation' project")
print("3. Compare runs for each retriever to analyze:")
print("   - Total cost per retriever")
print("   - Average latency per query")
print("   - Token usage patterns")
print("\nKey observations to look for:")
print("- Multi-Query Retriever: Higher cost (multiple LLM calls for query generation)")
print("- Contextual Compression: Additional cost for reranking")
print("- BM25: Lower cost (no embedding calls)")
print("- Ensemble: Combined cost of all component retrievers")
print("="*80)

In [ ]:
# Step 7: Visualize the results
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Retriever Performance Metrics Comparison', fontsize=16, fontweight='bold')

metrics_to_plot = list(comparison_df.columns)
colors = plt.cm.Set3(range(len(comparison_df)))

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 3, idx % 3]
    bars = ax.bar(comparison_df.index, comparison_df[metric], color=colors)
    ax.set_title(metric.replace('_', ' ').title(), fontweight='bold')
    ax.set_ylabel('Score')
    ax.set_ylim(0, 1)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# Step 6: Compile and visualize results
import pandas as pd
import matplotlib.pyplot as plt

# Create a comparison dataframe
comparison_data = []
for name, result in evaluation_results.items():
    row = {"Retriever": name}
    row.update(result)
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.set_index("Retriever")

print("Retriever Performance Comparison:")
print("="*80)
print(comparison_df.round(4))
print("="*80)

In [ ]:
# Step 5: Evaluate each retriever with Ragas metrics
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    context_entity_recall,
    noise_sensitivity,
    answer_relevancy,
    faithfulness
)

# Define metrics for evaluation
retriever_metrics = [
    context_precision,
    context_recall,
    context_entity_recall,
    noise_sensitivity
]

# Add answer quality metrics
answer_metrics = [
    answer_relevancy,
    faithfulness
]

all_metrics = retriever_metrics + answer_metrics

# Evaluate each retriever
evaluation_results = {}

print("Evaluating retrievers with Ragas...")
for name, dataset in retriever_datasets.items():
    print(f"\nEvaluating {name} retriever...")
    result = evaluate(dataset, metrics=all_metrics)
    evaluation_results[name] = result
    print(f"{name} evaluation complete!")

print("\nAll evaluations complete!")

In [ ]:
# Step 4: Get contexts from each retriever and create evaluation datasets
retrievers_to_evaluate = {
    "Naive": naive_retriever,
    "BM25": bm25_retriever,
    "Contextual Compression": compression_retriever,
    "Multi-Query": multi_query_retriever,
    "Parent Document": parent_document_retriever,
    "Ensemble": ensemble_retriever
}

print("Retrieving contexts for each retriever...")
for name, retriever in retrievers_to_evaluate.items():
    print(f"Processing {name} retriever...")
    contexts = get_retriever_contexts(retriever, questions)
    
    # Get answers from the retrieval chain
    answers = []
    for question in questions:
        # Construct the appropriate chain for each retriever
        if name == "Naive":
            chain = naive_retrieval_chain
        elif name == "BM25":
            chain = bm25_retrieval_chain
        elif name == "Contextual Compression":
            chain = contextual_compression_retrieval_chain
        elif name == "Multi-Query":
            chain = multi_query_retrieval_chain
        elif name == "Parent Document":
            chain = parent_document_retrieval_chain
        elif name == "Ensemble":
            chain = ensemble_retrieval_chain
        
        result = chain.invoke({"question": question})
        answers.append(result["response"].content)
    
    # Create dataset for this retriever
    retriever_datasets[name] = Dataset.from_dict({
        "user_input": questions,
        "retrieved_contexts": contexts,
        "response": answers,
        "reference": reference_answers,
        "reference_contexts": ground_truths
    })

print("All retriever contexts collected!")

In [ ]:
# Step 3: Prepare evaluation datasets for each retriever
from datasets import Dataset

# Create a function to get contexts from each retriever
def get_retriever_contexts(retriever, questions):
    contexts = []
    for question in questions:
        docs = retriever.invoke(question)
        contexts.append([doc.page_content for doc in docs])
    return contexts

# Prepare questions from testset
questions = testset_df['user_input'].tolist()
ground_truths = testset_df['reference_contexts'].tolist()
reference_answers = testset_df['reference'].tolist()

# Dictionary to store datasets for each retriever
retriever_datasets = {}

In [ ]:
# Step 2: Create a golden dataset using Ragas synthetic data generation
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Initialize generator LLM and embeddings
generator_llm = ChatOpenAI(model="gpt-4o-mini")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create test set generator
generator = TestsetGenerator.from_langchain(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings
)

# Generate test dataset
testset = generator.generate_with_langchain_docs(
    synthetic_usecase_data,
    test_size=10,
    distributions={simple: 0.4, reasoning: 0.4, multi_context: 0.2}
)

# Convert to DataFrame for easier viewing
testset_df = testset.to_pandas()
testset_df.head()